In [1]:
import pandas as pd

# Load the data
df = pd.read_csv('/kaggle/input/d/aicha222/predictionset/donnees_ventes_4.csv')

In [2]:
# Define the station mapping exactly as you want it
station_mapping = {
    1249949532: 1,
    1546539305: 2,
    -759695236: 3,
    1879002950: 4,
    1515917910: 5,
    138698241: 6,
    -1210215687: 7,
    1079293707: 8,
    358751465: 9,
    909091379: 10
}

# Print the station mapping
print("Station mapping:", station_mapping)


Station mapping: {1249949532: 1, 1546539305: 2, -759695236: 3, 1879002950: 4, 1515917910: 5, 138698241: 6, -1210215687: 7, 1079293707: 8, 358751465: 9, 909091379: 10}


In [3]:
# Apply the mapping to the 'Station' column
df['Station'] = df['Station'].map(station_mapping)

# Check the results
print(df['Station'].unique())


[ 2  6  1  5  4  7  3  9  8 10]


In [4]:
# Ensure 'DateTransaction' is in datetime format
df['DateTransaction'] = pd.to_datetime(df['DateTransaction'])

# Extract the date part of DateTransaction
df['DateOnly'] = df['DateTransaction'].dt.date

# Extract date features
df['Year'] = df['DateTransaction'].dt.year
df['Month'] = df['DateTransaction'].dt.month
df['Day'] = df['DateTransaction'].dt.day
df['Hour'] = df['DateTransaction'].dt.hour
df['Minute'] = df['DateTransaction'].dt.minute
df['Second'] = df['DateTransaction'].dt.second
df['DayOfWeek'] = df['DateTransaction'].dt.dayofweek
df['DayOfYear'] = df['DateTransaction'].dt.dayofyear
df['WeekOfYear'] = df['DateTransaction'].dt.isocalendar().week

# Drop the original DateTransaction column if not needed
df.drop(columns=['DateTransaction'], inplace=True)

# Display the first few rows of the dataframe
df.head()


,Station,TransactionNumber,Pump,Product,TransactionVolume,TransactionPrice,DateOnly,Year,Month,Day,Hour,Minute,Second,DayOfWeek,DayOfYear,WeekOfYear
0,2,1570,6,2,21.83,300.00,2023-10-01,2023,10,1,12,7,38,6,274,39
1,6,7802,9,1,2.15,30.01,2023-06-06,2023,6,6,7,52,30,1,157,23
2,1,7447,7,2,33.60,457.63,2023-02-03,2023,2,3,8,50,11,4,34,5
3,1,8475,4,2,14.68,200.00,2023-02-03,2023,2,3,17,29,35,4,34,5
4,1,1540,13,2,43.38,600.00,2023-11-26,2023,11,26,3,8,17,6,330,47


In [5]:
# Aggregate transaction volume for each station by day
Station_Day_Aggregation = df.groupby(['Station', 'DateOnly'])['TransactionVolume'].sum().reset_index()

# Merge aggregated data with cyclic features
# Drop duplicates to ensure unique rows
cyclic_features = df[['Station', 'DateOnly',
                      'Month',
                      'Day',
                      'DayOfWeek',
                      'DayOfYear',
                      'WeekOfYear']].drop_duplicates()

# Merge the aggregated transaction volume with cyclic features
df_aggregated_with_cyclic = pd.merge(Station_Day_Aggregation, cyclic_features, on=['Station', 'DateOnly'], how='left')

print(df_aggregated_with_cyclic)

      Station    DateOnly  TransactionVolume  Month  Day  DayOfWeek  \
0           1  2023-01-01           29737.21      1    1          6   
1           1  2023-01-02           40177.30      1    2          0   
2           1  2023-01-03           38395.26      1    3          1   
3           1  2023-01-04           36586.46      1    4          2   
4           1  2023-01-05           35959.38      1    5          3   
...       ...         ...                ...    ...  ...        ...   
5886       10  2024-08-08            1365.88      8    8          3   
5887       10  2024-08-09            1766.66      8    9          4   
5888       10  2024-08-10            1125.18      8   10          5   
5889       10  2024-08-11             917.68      8   11          6   
5890       10  2024-08-12             269.58      8   12          0   

      DayOfYear  WeekOfYear  
0             1          52  
1             2           1  
2             3           1  
3             4           1

In [6]:
for column_name in df_aggregated_with_cyclic.columns:
    column = df_aggregated_with_cyclic[column_name]
    # Get the count of Zeros in column
    count = (column == 0).sum()
    print(column_name, ':', count)

Station : 0
DateOnly : 0
TransactionVolume : 0
Month : 0
Day : 0
DayOfWeek : 848
DayOfYear : 0
WeekOfYear : 0


In [7]:
import pandas as pd

# Convert 'Date' column to datetime if it's not already in datetime format
df_aggregated_with_cyclic['DateOnly'] = pd.to_datetime(df_aggregated_with_cyclic['DateOnly'])

# Filter the dataframe for the specific date and station
filtered_df = df_aggregated_with_cyclic[(df_aggregated_with_cyclic['DateOnly'] == '2024-06-11') & (df_aggregated_with_cyclic['Station'] == 1)]

# Display the information for the filtered row(s)
print(filtered_df)


     Station   DateOnly  TransactionVolume  Month  Day  DayOfWeek  DayOfYear  \
527        1 2024-06-11           28922.26      6   11          1        163   

     WeekOfYear  
527          24  


In [8]:
filtered_data = df_aggregated_with_cyclic[
    (df_aggregated_with_cyclic['DateOnly'] == pd.to_datetime('2024-07-15')) &
    (df_aggregated_with_cyclic['Station'] == 4 )
]
print(filtered_data)

      Station   DateOnly  TransactionVolume  Month  Day  DayOfWeek  DayOfYear  \
2322        4 2024-07-15            5864.06      7   15          0        197   

      WeekOfYear  
2322          29  
